# 科学技術計算 5 レポート


In [156]:
import numpy as np
import scipy

from numpy.linalg import inv, norm, solve, det, matrix_rank, cond, pinv, lstsq, eig, eigh, eigvals, eigvalsh
from numpy import diag
rng = np.random.default_rng()

import matplotlib.pyplot as plt
from matplotlib import rcParams
rcParams["savefig.bbox"] = "tight"

from typing import List, Optional, Union, Tuple

import sklearn
from sklearn.decomposition import PCA
from sklearn.datasets import fetch_olivetti_faces

from matplotlib.pyplot import imshow
from typing import Tuple, List, Optional, Union
from tqdm.auto import tqdm



## 課題05-1

In [157]:
#レイリー商反復について

#関数
def rayleigh_quotient_iteration(A: np.ndarray,mu: float, u, tol = 1e-12,n = np.ndarray, maxiter: int = 5000,) -> Tuple[float, np.ndarray]:
    """Rayleigh quotioent iteration for computing the largest eigenvalue and its eigenvector of A

    Args:
        A (np.ndarray): nxn matrix A
        mu (float): initial eigenvalue
        u (np.ndarray): n-d initial eigenvector
        maxiter (int, optional): max iterations. Defaults to 50.

    Returns:
        Tuple[float, np.ndarray]: the largest eigenvalue and its eigenvector
    """
    #入力 行列Aが対称かどうか muがシフト値uは
    assert A.ndim == 2
    assert A.shape[0] == A.shape[1]
    n = A.shape[0]
    assert len(u) == n

    u_pre = u.copy()

    #レイリー商反復
    for i in range(maxiter):

        try:
            u = solve(A - mu * np.eye(n), u)    #シフト値muを用いて連立方程式を解く
        except:
            break
        
        u /= norm(u)    #正規化
        mu = u.T @ A @ u

        with np.printoptions(formatter={'float': '{: 12.12f}'.format}, suppress=True, linewidth=100):
            print(f"{i:2d}", u)

        if np.allclose(u, u_pre) or np.allclose(u, -u_pre):         #uの値がめっちゃちかくなった→近似
            break
        u_pre = u.copy()
        
    return mu, u



In [ ]:
#ハウスホルダー法
def householder_step( A: np.ndarray, k: int) -> Tuple[np.ndarray, np.ndarray]:
    """k-th step of Householder transform from nxn symmetric to tridiagonal

    Args:
        A (np.ndarray): an nxn symmetric matrix
        k (int): step

    Returns:
        Tuple[np.ndarray, np.ndarray]: transformed matrix QAQ after k-th step, and the transfrom Q
    """
    assert A.ndim == 2  #2次元であるかチェック
    assert A.shape[0] == A.shape[1] #正方行列かどうかチェック

    n = A.shape[0]
    assert 0 <= k < n - 2

    x = A[k + 1:, k]  # Aのk列目のk+1行目以降の部分を取得(縦の部分)
    y = np.zeros_like(x) #全部ゼロの配列
    # y[0] = - np.sign(A[k + 1, k]) * norm(x)
    y[0] = - np.sign(x[0]) * norm(x)  # ベクトルyの先頭要素を設定

    # print("x", x)
    # print("y", y)

    u = (x - y) / norm(x - y)  # 単位ベクトルuを計算
    H = np.eye(len(x)) - 2 * np.outer(u, u)  # ハウスホルダー行列Hを計算

    Q = np.eye(n)   #単位行列
    Q[k + 1:, k + 1:] = H  # QにHを組み込む

    QAQ = Q @ A @ Q  # 変換後の行列を計算
    return QAQ, Q

def householder_transform(A: np.ndarray,) -> Tuple[np.ndarray, np.ndarray]:
    """Householder transform from nxn symmetric to tridiagonal

    Args:
        A (np.ndarray): an nxn symmetric matrix

    Returns:
        Tuple[np.ndarray, np.ndarray]: tridiagonal Atri and orthogonal Qhh
                                       so that Qhh A Qhh == Atri
    """
    assert A.ndim == 2
    assert A.shape[0] == A.shape[1]
    n = A.shape[0]

    Atri = A.copy()
    Qhh = np.eye(n)
    for k in range(n - 2):
        # print(f"{k}-th step")
        Atri, Q = householder_step(Atri, k)
        Qhh = Qhh @ Q  # 直交行列を更新
        # print("Atri\n", Atri, sep='')
        # print("Q\n", Q, sep='')
        # print("Qhh\n", Qhh, sep='')
    return Atri, Qhh


In [159]:
#ぎぶんす変換
def givens_r_matrix(xp_xq: np.ndarray) -> np.ndarray:
    """generate a matrix of Givens rotation

    Args:
        xp_xq (ndarray): two values xp, xq

    Returns:
        ndarray: 2D rotation matrix
    """
    assert len(xp_xq) == 2

    xp, xq = xp_xq[0], xp_xq[1]

    # denominator = np.sqrt(xp**2 + xq**2)
    denominator = norm(xp_xq)
    c = xp / denominator
    s = -xq / denominator

    R = np.array([
        [c, s],
        [-s, c]
    ])
    return R

def qr_decom_by_givens(Atri_org: np.ndarray,) -> Tuple[np.ndarray, np.ndarray]:
    """QR decomposition of a tridiagonal matrix by Givens rotation

    Args:
        Atri_org (np.ndarray): an nxn tridiagonal matrix

    Returns:
        Tuple[np.ndarray, np.ndarray]: orthogonal Q and upper traiangular R so that A == QR
    """
    assert len(Atri_org.shape) == 2
    assert Atri_org.shape[0] == Atri_org.shape[1]
    # assert AAtri_org should be tridiagonal
    n = Atri_org.shape[0]

    Atri = Atri_org.copy()
    Q = np.eye(n)

    for k in range(n - 1):
        p, q = k, k + 1
        Rot2x2 = givens_r_matrix(Atri[[p, q]][:, p])
        Atri[[p, q]] = Rot2x2.T @ Atri[[p, q]]
        Q[:, [p, q]] = Q[:, [p, q]] @ Rot2x2

    R = Atri
    return Q, R

In [ ]:
#QR法関数
def eigh_by_qr_method(A0: np.ndarray, maxiter: int = 1000,) -> Tuple[np.ndarray, np.ndarray]:
    """
    Computing eigenvalues and eigenvectors by QR decompoition

    Args:
        A0 (np.ndarray): nxn matrix
        maxiter (int, optional): max iterations. Defaults to 1000.
        qr_decomp (callable, optional): QR decomposition solver. Defaults to np.linalg.qr.

    Returns:
        Tuple[np.ndarray, np.ndarray]: n-d vector of eigenvalues and orthogonal matrix with eigenvector columns
    """
    assert A0.ndim == 2
    assert A0.shape[0] == A0.shape[1]
    n = A0.shape[0]
    A = A0.copy()

    Qall = np.eye(n)
    diagonals_pre = diag(A)
    #ハウスホルダー変換
    Atri, Qhh = householder_transform(A)
    Qall = Qhh
    Atri_org = Atri.copy()

    for i in range(maxiter):
        
        Q, R = qr_decom_by_givens(Atri_org)
        Atri_org = R @ Q
        Qall = Qall @ Q

        diagonals = diag(Atri_org)
        # with np.printoptions(formatter={'float': '{: 12.8f}'.format}, suppress=True, linewidth=100):
        #     print(f"     diagonals {i:2d}", diagonals)
        #     print(f"superdiagonals {i:2d}", superdiagonals)

        if (np.allclose(diagonals, diagonals_pre)):
            # with np.printoptions(formatter={'float': '{: 12.8f}'.format}, suppress=True, linewidth=100):
            #     print("A\n", A, sep='')
            break

        diagonals_pre = diagonals.copy()

    return diagonals, Qall


In [161]:
#今回のやつ
def eigh_via_qr_rayleigh( A: np.ndarray,) -> Tuple[np.ndarray, np.ndarray]:
    # print("A =", A)
    assert A.ndim == 2
    assert A.shape[0] == A.shape[1]
    n = A.shape[0]

    
    eigenvalues_approx, Qqr  = eigh_by_qr_method(A) #これでAがぎぶんす変換で更新しながら固有値と固有べくとるを出力できたってことかな

    eigenvalues = np.zeros(n)
    eigenvectors = np.zeros((n, n))

    for k in range(n):
        mu0 = eigenvalues_approx[k]
        u0 = Qqr[k, :]


        mu_refined, u_refined = rayleigh_quotient_iteration(A, mu0, u0)

        eigenvalues[k] = mu_refined
        eigenvectors[:, k] = u_refined

    return eigenvalues, eigenvectors


In [164]:
#main
#実行
for i in tqdm(range(10)):
    n = rng.integers(low = 2, high = 10)
    A = rng.random(size=(n, n))

    #対称行列
    A = A.T @ A
    print("行列 A:")
    print(A)

    d, U = eigh_via_qr_rayleigh(A)
    eigvals_np, eigvecs_np = np.linalg.eigh(A)


    with np.printoptions(formatter={'float': '{: 12.8f}'.format}, suppress=True, linewidth=100):
        d_np, U_np = np.linalg.eigh(A)
        U_np = U_np[:, ::-1]
        d_np = d_np[::-1]

        # print("d_np", d_np)
        # print("d   ", d)
        print("d == d_np", np.allclose(d, d_np))

        # print("U_np\n", U_np, sep='')
        # print("U\n", U, sep='')

        for i in range(len(U)):
            print(
                f"column {i}: U[{i}] == U_np[{i}] ?",
                np.allclose(U[:, i], U_np[:, i]) or np.allclose(U[:, i], -U_np[:, i])
            )



  0%|          | 0/10 [00:00<?, ?it/s]

行列 A:
[[1.74881558 1.2612964  1.86714695 1.0681152  1.3743124 ]
 [1.2612964  1.25604793 1.52253085 0.88827348 1.33570167]
 [1.86714695 1.52253085 3.08865279 1.83829044 1.94646883]
 [1.0681152  0.88827348 1.83829044 1.11428283 1.07432175]
 [1.3743124  1.33570167 1.94646883 1.07432175 1.87076195]]
 0 [ 0.425248547309  0.361312555582  0.606096376477  0.353688110772  0.442909486138]
 1 [-0.425248547309 -0.361312555582 -0.606096376477 -0.353688110772 -0.442909486138]
 0 [ 0.314287878224  0.455521125509 -0.561483305650 -0.430883071300  0.439089865700]
 1 [ 0.314294458231  0.455528150086 -0.561496049376 -0.430866449330  0.439077883047]
 2 [ 0.314294458231  0.455528150086 -0.561496049376 -0.430866449330  0.439077883047]
 0 [-0.757589367149 -0.018754520684  0.072792120474 -0.006618794758  0.648354931723]
 1 [-0.757576134284 -0.018792832827  0.072796514857 -0.006622049514  0.648368757793]
 2 [-0.757576134284 -0.018792832827  0.072796514857 -0.006622049514  0.648368757793]
 0 [-0.372402199161  0.

## 課題05-3